In [ ]:
!pip install openai arxiv
!rm *.json*
# !wget https://raw.githubusercontent.com/Papercopilot/paperlists/main/iclr/iclr2025.json
!wget https://raw.githubusercontent.com/Papercopilot/paperlists/main/iclr/iclr2024.json
!wget https://raw.githubusercontent.com/Papercopilot/paperlists/main/iclr/iclr2023.json
!wget https://raw.githubusercontent.com/Papercopilot/paperlists/main/iclr/iclr2022.json
!wget https://raw.githubusercontent.com/Papercopilot/paperlists/main/iclr/iclr2021.json

!wget https://github.com/papercopilot/paperlists/raw/refs/heads/main/nips/nips2024.json
!wget https://github.com/papercopilot/paperlists/raw/refs/heads/main/nips/nips2023.json
!wget https://github.com/papercopilot/paperlists/raw/refs/heads/main/nips/nips2022.json
!wget https://github.com/papercopilot/paperlists/raw/refs/heads/main/nips/nips2021.json
!wget https://github.com/papercopilot/paperlists/raw/refs/heads/main/nips/nips2020.json

!wget https://github.com/papercopilot/paperlists/raw/refs/heads/main/icml/icml2024.json
!wget https://github.com/papercopilot/paperlists/raw/refs/heads/main/icml/icml2023.json
!wget https://github.com/papercopilot/paperlists/raw/refs/heads/main/icml/icml2022.json
!wget https://github.com/papercopilot/paperlists/raw/refs/heads/main/icml/icml2021.json



!wget https://raw.githubusercontent.com/Papercopilot/paperlists/main/siggraph/siggraph2023.json


from IPython.display import clear_output

clear_output()


In [ ]:
from google.colab import drive

# 挂载 Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm  # 导入tqdm

In [ ]:
loadFile="iclr2024"

with open(loadFile+'.json','r') as file:
  paperData=json.load(file)

df = pd.DataFrame(paperData)
df.head()

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key='sk-uGDpj82N23f70b22B0ECT3BLBKFJ4cD20d910aDD4df69e1D',
    base_url='https://c-z0-api-01.hash070.com/v1'
)

print(df["status"].unique())
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant. 简洁地返回结果，不要包含无关内容。返回python友好源代码（非markdown格式）"},
    {"role": "user", "content": f"从下面这个array中，筛选正常的paper状态（非拒稿撤回等）{df['status'].unique()}，返回一个list。"}
  ]
)

print(completion.choices[0].message.content)

['Withdraw' 'Reject' 'Poster' 'Spotlight' 'Oral' 'Desk Reject']
['Poster', 'Spotlight', 'Oral']


In [ ]:
# 提取返回的结果并转换为 Python 列表
response_content = completion.choices[0].message.content.strip()
normal_statuses = eval(response_content)  # 注意：eval 可能有安全风险，请确保 API 返回的内容是可信的

# 使用返回的正常状态列表筛选 DataFrame
df = df[df['status'].isin(normal_statuses)]


In [ ]:
# df = df[df['status'].isin(['Poster', 'Oral', 'Spotlight'])]
# df = df[df['status'].isin(['Active'])]
# df = df[df['status'].isin(['Poster', 'Journal', 'Highlighted'])]
# df['primary_area'].unique()

#SIG 无需

df['sess'].unique()


KeyError: 'sess'

获取Abstract内容

In [ ]:
import arxiv
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# 定义函数，从 arXiv 获取摘要
def get_abstract_from_arxiv(title):
    search = arxiv.Search(
        query=f"ti:{title}",
        max_results=1,
        sort_by=arxiv.SortCriterion.Relevance
    )
    try:
        for result in search.results():
            return result.summary
    except Exception as e:
        print(f"Error fetching abstract for title '{title}': {e}")
        return None

# 使用多线程批量获取摘要
def fetch_all_abstracts(titles, max_workers=10):
    abstracts = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(get_abstract_from_arxiv, title) for title in titles]
        for future in tqdm(as_completed(futures), total=len(titles), desc="Fetching abstracts"):
            abstracts.append(future.result())
    return abstracts



# 多线程获取摘要
df['abstract'] = fetch_all_abstracts(df['title'])
print(df)

<ipython-input-27-45b45ebe061b>:14: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():
Fetching abstracts: 100%|██████████| 2260/2260 [15:48<00:00,  2.38it/s]

              id                                              title track  \
5     02f3mUtqnM  Hybrid LLM: Cost-Efficient and Quality-Aware Q...  main   
10    06lrITXVAx                  Dropout Enhanced Bilevel Training  main   
14    09iOdaeOzp  Sheared LLaMA: Accelerating Language Model Pre...  main   
15    09xFexjhqE  AutoLoRa: An Automated Robust Fine-Tuning Fram...  main   
18    0BqyZSWfzo  One-shot Empirical Privacy Estimation for Fede...  main   
...          ...                                                ...   ...   
7395  ztpy1gsUpT  Enhancing Small Medical Learners with Privacy-...  main   
7398  zwMfg9PfPs  Out-of-Variable Generalisation for Discriminat...  main   
7399  zwU9scoU4A  Learning Mean Field Games on Sparse Graphs: A ...  main   
7403  zyBJodMrn5  On the generalization capacity of neural netwo...  main   
7405  zzqn5G9fjn  Breaking Physical and Linguistic Borders: Mult...  main   

         status                                           keywords  \
5    

什么年代了还在生成embedding，直接构造prompt！

In [ ]:
# 生成 text prompt
def generate_text_prompt(df):
    # 确保只关注 title 和 abstract 列
    df = df[['title', 'abstract']].dropna()  # 去除缺失值
    prompt = ""
    for idx, row in enumerate(df.itertuples(index=False), start=1):
        title, abstract = row.title, row.abstract
        prompt += f"{idx}. Title: {title}\n   Abstract: {abstract}\n\n"
    return prompt.strip()  # 去除末尾多余的换行符

# 调用函数生成 prompt
text_prompt = generate_text_prompt(df)

# 打印结果
print(text_prompt)

1. Title: AutoLoRa: An Automated Robust Fine-Tuning Framework
   Abstract: Predicting and reasoning about the future lie at the heart of many
time-series questions. For example, goal-conditioned reinforcement learning can
be viewed as learning representations to predict which states are likely to be
visited in the future. While prior methods have used contrastive predictive
coding to model time series data, learning representations that encode
long-term dependencies usually requires large amounts of data. In this paper,
we introduce a temporal difference version of contrastive predictive coding
that stitches together pieces of different time series data to decrease the
amount of data required to learn predictions of future events. We apply this
representation learning method to derive an off-policy algorithm for
goal-conditioned RL. Experiments demonstrate that, compared with prior RL
methods, ours achieves $2 \times$ median improvement in success rates and can
better cope with stochas

In [ ]:
output_path =f'/content/drive/MyDrive/{loadFile}.csv'

# 将 DataFrame 保存为 CSV 文件
df.to_csv(output_path, index=False)

print(f"文件已保存到：{output_path}")

文件已保存到：/content/drive/MyDrive/iclr2024.csv


In [ ]:
df[df['title'].str.contains("MOF", na=False)]

,id,title,track,status,keywords,primary_area,author,authorids,aff,aff_domain,...,confidence_avg,soundness_avg,contribution_avg,presentation_avg,replies_avg,corr_rating_confidence,project,github,site,abstract
46,0VBsoluxR2,MOFDiff: Coarse-grained Diffusion for Metal-Or...,main,Poster,Materials design;diffusion model;metal-organic...,"applications to physical sciences (physics, ch...",Xiang Fu;Tian Xie;Andrew Scott Rosen;Tommi S. ...,~Xiang_Fu4;~Tian_Xie2;~Andrew_Scott_Rosen1;~To...,Massachusetts Institute of Technology;Microsof...,mit.edu;microsoft.com;berkeley.edu;microsoft.com,...,3.50,2.75,2.5,3.25,20,0.000000,,,https://iclr.cc/virtual/2024/poster/19615,The great success of Large Language Models (LL...
3092,QQYpgReSRk,MOFI: Learning Image Representations from Nois...,main,Poster,Image representation;image embedding;image bas...,"representation learning for computer vision, a...",Wentao Wu;Aleksei Timofeev;Chen Chen;Bowen Zha...,~Wentao_Wu3;~Aleksei_Timofeev1;~Chen_Chen38;~B...,Apple;Apple;Apple;Apple;Apple;Apple;Apple Inc.,apple.com;apple.com;apple.com;apple.com;apple....,...,4.25,3.00,2.5,3.50,25,-0.132453,,,https://iclr.cc/virtual/2024/poster/18677,We propose Hyper-Dimensional Function Encoding...


In [ ]:
df[df['title'].str.contains("MOF", na=False)].to_markdown()
# 解析这个表格的内容，我想知道每篇paper的标题，机构（简略），Abstract简要概括，评分


'|      | id         | title                                                                   | track   | status   | keywords                                                                                                | primary_area                                                                       | author                                                                                                                       | authorids                                                                                                                                         | aff                                                                                                                  | aff_domain                                                            | position                                                                                    | rating   | confidence   | soundness   | contribution   | presentation   |   rating_avg |   confidence_avg |   soundness_avg |   con